In [1]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

In [2]:
!pip install autokeras


In [3]:
import numpy as np
import pandas as pd
import re
import string
import math

import numpy as np
import tensorflow as tf
from sklearn.datasets import load_files
from tensorflow.keras.models import Sequential

import autokeras as ak


In [4]:
data = pd.read_csv('../input/resume-dataset/Resume/Resume.csv', usecols=['Resume_str','Category'])

In [5]:
data.head()

In [6]:
contractions_dict = {"ain't": "are not","'s":" is","aren't": "are not"}
# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)
# Expanding Contractions in the reviews
data['Resume_str']=data['Resume_str'].apply(lambda x:expand_contractions(x))

In [7]:
data.head()

In [8]:
data["Resume_str"] = data["Resume_str"].str.lower()


In [9]:
data['Resume_str'] = data['Resume_str'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '' , x))


In [10]:
data['Resume_str'] = data['Resume_str'].apply(lambda x: re.sub('W*dw*','',x))


In [11]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stop_words.add('subject')
stop_words.add('http')
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])
data['Resume_str'] = data['Resume_str'].apply(lambda x: remove_stopwords(x))

In [12]:
data['Resume_str'] = data['Resume_str'].apply(lambda x: re.sub('b[w-]+?@w+?.w{2,4}b', 'emailadd',x))
#url
data['Resume_str'] = data['Resume_str'].apply(lambda x:re.sub('(http[s]?S+)|(w+.[A-Za-z]{2,4}S*)', 'urladd', x))

In [13]:
#stemming
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])
data["Resume_str"] = data["Resume_str"].apply(lambda x: stem_words(x))


In [14]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])
data["Resume_str"] = data["Resume_str"].apply(lambda text: lemmatize_words(text))

In [15]:
#remove space
data["Resume_str"] = data["Resume_str"].apply(lambda x: re.sub(' +', ' ', x))


In [16]:
data.iloc[1,0]

In [17]:
x=data['Resume_str'].values
y=data['Category'].values

In [18]:
from  sklearn.model_selection import train_test_split


In [19]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [25]:
clf = ak.TextClassifier(
    overwrite=True, max_trials=1
)  # It only tries 1 model as a quick demo.
# Feed the text classifier with training data.
clf.fit(x_train, y_train, epochs=30)
# Predict with the best model.


In [26]:
predicted_y = clf.predict(x_test)
print(clf.evaluate(x_test, y_test))

In [32]:
predicted_y[2][0]

In [34]:
y_test[2]